In [1]:
#Import required libraries
import numpy as np
import pandas as pd
import pandas as pd
from math import pow, sqrt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
import warnings
warnings.filterwarnings('ignore')

d:\softwares\python3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#Read data to pandas dataframe
ratings = pd.read_csv('dataset.csv') 

In [3]:
#print dataframe
ratings

,user_id,sex,age,occupation,movie title,movie_id,rating,Musical,unknown,Action,...,Fantasy,Film-Noir,Horror,Mystery,Romance,Thriller,War,Sci-Fi,Western,unix_timestamp
0,173,M,56,other,Evita (1996),289,4,0,0,1,...,0,0,0,0,1,0,0,0,0,877556988
1,607,F,49,healthcare,Raiders of the Lost Ark (1981),174,3,0,0,0,...,0,0,0,0,1,1,0,0,0,883879516
2,600,M,34,programmer,Natural Born Killers (1994),53,4,0,0,0,...,0,0,0,0,0,0,0,0,0,888452563
3,104,M,27,student,Beverly Hills Ninja (1997),456,3,0,0,0,...,0,0,0,0,0,0,0,0,0,888465853
4,486,M,39,educator,"Frighteners, The (1996)",123,3,0,0,0,...,0,0,0,0,1,0,0,0,0,879875278
5,372,F,25,student,Scream (1996),288,5,0,0,0,...,0,0,0,0,0,0,0,0,0,876869066
6,94,M,26,student,Casablanca (1942),483,5,0,0,0,...,0,0,0,0,0,0,0,0,0,885870115
7,188,M,42,student,Speed (1994),568,4,0,0,1,...,0,0,0,0,0,0,1,0,0,875072583
8,297,F,29,educator,Alien (1979),183,4,0,0,0,...,0,0,0,0,0,0,0,0,0,875238984
9,329,M,48,educator,Singin' in the Rain (1952),705,3,0,0,0,...,0,0,0,0,0,1,0,0,0,891656347


In [4]:
#Rearranging dataframe
moviemat = ratings.pivot_table(index='user_id',columns='movie_id',values='rating').fillna(0)
moviemat.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1669,1670,1671,1672,1674,1677,1678,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#converting dataset to CSR matrix
mat = csr_matrix(moviemat.values)

In [6]:
#Using KNN algorithm with cosine similarity
model_knn=NearestNeighbors(metric="cosine",algorithm="brute")
model_knn.fit(mat)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [7]:
#Testing model for index 0
query_index=0

recommended=[] 
distance,indice=model_knn.kneighbors(moviemat.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)
print("Distance:",distance," Indice :",indice)
 
print(distance.flatten())
print(len(distance.flatten()))
 
for i in range(0,len(distance.flatten())):
    if i==0:
        print("Recommendation for {0}:\n".format(moviemat.iloc[query_index,:]))
    else:
        print("{0}: {1}, with calculated distance = {2}:".format(i,moviemat.index[indice.flatten()[i]],distance.flatten()[i]))
        recommended.append(moviemat.index[indice.flatten()[i]])


    print(recommended)

Distance: [[1.55431223e-15 4.94422473e-01 5.12465121e-01 5.25721913e-01
  5.27326314e-01 5.44148221e-01]]  Indice : [[  0 434 267 915 428 456]]
[1.55431223e-15 4.94422473e-01 5.12465121e-01 5.25721913e-01
 5.27326314e-01 5.44148221e-01]
6
Recommendation for movie_id
1       5.0
2       3.0
3       4.0
4       3.0
5       3.0
6       5.0
7       4.0
8       1.0
9       5.0
10      3.0
11      2.0
12      5.0
13      5.0
14      0.0
15      5.0
16      0.0
17      3.0
18      4.0
19      5.0
20      4.0
21      1.0
22      4.0
23      4.0
24      3.0
25      4.0
26      3.0
27      0.0
28      0.0
29      1.0
30      3.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655    0.0
1656    0.0
1658    0.0
1659    0.0
1660    0.0
1661    0.0
1662    0.0
1663    0.0
1664    0.0
1665    0.0
1668    0.0
1669    0.0
1670    0.0
1671    0.0
1672    0.0
1674    0.0
1677    0.0
1678    0.0
1680    0.0
1681    0.0
1682    0.0
N

In [8]:
#Writing recommendations to file
myfile = open('recommended.txt', 'w')

for userid in range(0,943):
    myfile.write("\n") 
    query_index=userid
    myfile.write(str(query_index+1))
    recommended=[] 
    distance,indice=model_knn.kneighbors(moviemat.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)
    print("Distance :",distance," Indice :",indice)

    print(distance.flatten())
    print(len(distance.flatten()))

    for i in range(0,len(distance.flatten())):
        if i==0:
            print("Recommendation for {0}:\n".format(moviemat.iloc[query_index,:]))
        else:
            print("{0}: {1}, with distance of {2}:".format(i,moviemat.index[indice.flatten()[i]],distance.flatten()[i]))
            recommended.append(moviemat.index[indice.flatten()[i]])
    print(recommended)
    
    for item in recommended:
        myfile.write(",%s" % item)
     

Distance : [[1.55431223e-15 4.94422473e-01 5.12465121e-01 5.25721913e-01
  5.27326314e-01 5.44148221e-01]]  Indice : [[  0 434 267 915 428 456]]
[1.55431223e-15 4.94422473e-01 5.12465121e-01 5.25721913e-01
 5.27326314e-01 5.44148221e-01]
6
Recommendation for movie_id
1       5.0
2       3.0
3       4.0
4       3.0
5       3.0
6       5.0
7       4.0
8       1.0
9       5.0
10      3.0
11      2.0
12      5.0
13      5.0
14      0.0
15      5.0
16      0.0
17      3.0
18      4.0
19      5.0
20      4.0
21      1.0
22      4.0
23      4.0
24      3.0
25      4.0
26      3.0
27      0.0
28      0.0
29      1.0
30      3.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655    0.0
1656    0.0
1658    0.0
1659    0.0
1660    0.0
1661    0.0
1662    0.0
1663    0.0
1664    0.0
1665    0.0
1668    0.0
1669    0.0
1670    0.0
1671    0.0
1672    0.0
1674    0.0
1677    0.0
1678    0.0
1680    0.0
1681    0.0
1682    0.0


Distance : [[1.11022302e-16 6.12357690e-01 6.39982790e-01 6.40922942e-01
  6.73547225e-01 6.98675229e-01]]  Indice : [[ 26  31 402  49  68 466]]
[1.11022302e-16 6.12357690e-01 6.39982790e-01 6.40922942e-01
 6.73547225e-01 6.98675229e-01]
6
Recommendation for movie_id
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       4.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655    0.0
1656    0.0
1658    0.0
1659    0.0
1660    0.0
1661    0.0
1662    0.0
1663    0.0
1664    0.0
1665    0.0
1668    0.0
1669    0.0
1670    0.0
1671    0.0
1672    0.0
1674    0.0
1677    0.0
1678    0.0
1680    0.0
1681    0.0
1682    0.0


Distance : [[2.22044605e-16 6.87134565e-01 6.88736636e-01 6.89693970e-01
  6.95537995e-01 7.00890339e-01]]  Indice : [[ 52 702 437 347 568 136]]
[2.22044605e-16 6.87134565e-01 6.88736636e-01 6.89693970e-01
 6.95537995e-01 7.00890339e-01]
6
Recommendation for movie_id
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       3.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      5.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      3.0
25      4.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655    0.0
1656    0.0
1658    0.0
1659    0.0
1660    0.0
1661    0.0
1662    0.0
1663    0.0
1664    0.0
1665    0.0
1668    0.0
1669    0.0
1670    0.0
1671    0.0
1672    0.0
1674    0.0
1677    0.0
1678    0.0
1680    0.0
1681    0.0
1682    0.0


Distance : [[0.         0.5874203  0.60959392 0.61404415 0.62988702 0.64199159]]  Indice : [[ 78  62 661 490 936 734]]
[0.         0.5874203  0.60959392 0.61404415 0.62988702 0.64199159]
6
Recommendation for movie_id
1       4.0
2       0.0
3       0.0
4       0.0
5       0.0
6       4.0
7       5.0
8       0.0
9       0.0
10      5.0
11      0.0
12      0.0
13      3.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      5.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655    0.0
1656    0.0
1658    0.0
1659    0.0
1660    0.0
1661    0.0
1662    0.0
1663    0.0
1664    0.0
1665    0.0
1668    0.0
1669    0.0
1670    0.0
1671    0.0
1672    0.0
1674    0.0
1677    0.0
1678    0.0
1680    0.0
1681    0.0
1682    0.0
Name: 79, Length: 1649, dtype: float64:

1: 63, wit

Distance : [[0.         0.41377343 0.4531403  0.46649818 0.4710253  0.47109955]]  Indice : [[104 919 841 826 819 808]]
[0.         0.41377343 0.4531403  0.46649818 0.4710253  0.47109955]
6
Recommendation for movie_id
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655    0.0
1656    0.0
1658    0.0
1659    0.0
1660    0.0
1661    0.0
1662    0.0
1663    0.0
1664    0.0
1665    0.0
1668    0.0
1669    0.0
1670    0.0
1671    0.0
1672    0.0
1674    0.0
1677    0.0
1678    0.0
1680    0.0
1681    0.0
1682    0.0
Name: 105, Length: 1649, dtype: float64:

1: 920, w

...

1: 654, with distance of 0.6218429566373251:
2: 883, with distance of 0.6298972583518827:
3: 738, with distance of 0.6340748511550716:
4: 43, with distance of 0.6397805720804606:
5: 465, with distance of 0.6401275135606628:
[654, 883, 738, 43, 465]
Distance : [[0.         0.64232193 0.65409629 0.66311386 0.68288416 0.68372781]]  Indice : [[838 633 485 180 623 539]]
[0.         0.64232193 0.65409629 0.66311386 0.68288416 0.68372781]
6
Recommendation for movie_id
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       2.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655    0.0
1656    0.0
1658    0.0
1659    0.0
1660  

Distance : [[2.22044605e-16 4.85492058e-01 4.86605394e-01 4.95590914e-01
  5.03465814e-01 5.08398849e-01]]  Indice : [[862 723 205 283 610  99]]
[2.22044605e-16 4.85492058e-01 4.86605394e-01 4.95590914e-01
 5.03465814e-01 5.08398849e-01]
6
Recommendation for movie_id
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655    0.0
1656    0.0
1658    0.0
1659    0.0
1660    0.0
1661    0.0
1662    0.0
1663    0.0
1664    0.0
1665    0.0
1668    0.0
1669    0.0
1670    0.0
1671    0.0
1672    0.0
1674    0.0
1677    0.0
1678    1.0
1680    2.0
1681    0.0
1682    0.0


Distance : [[0.         0.52498692 0.53200319 0.53446962 0.54530407 0.5466895 ]]  Indice : [[885 932 681 863 428 275]]
[0.         0.52498692 0.53200319 0.53446962 0.54530407 0.5466895 ]
6
Recommendation for movie_id
1       4.0
2       4.0
3       3.0
4       3.0
5       3.0
6       0.0
7       0.0
8       0.0
9       5.0
10      3.0
11      5.0
12      0.0
13      0.0
14      0.0
15      3.0
16      0.0
17      4.0
18      0.0
19      0.0
20      2.0
21      0.0
22      4.0
23      0.0
24      4.0
25      0.0
26      4.0
27      2.0
28      4.0
29      1.0
30      0.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655    0.0
1656    0.0
1658    0.0
1659    0.0
1660    0.0
1661    0.0
1662    0.0
1663    0.0
1664    0.0
1665    0.0
1668    0.0
1669    0.0
1670    0.0
1671    0.0
1672    0.0
1674    0.0
1677    0.0
1678    0.0
1680    0.0
1681    0.0
1682    0.0
Name: 886, Length: 1649, dtype: float64:

1: 933, w

Distance : [[5.55111512e-16 5.88137910e-01 6.26111225e-01 6.26262626e-01
  6.26469514e-01 6.26602733e-01]]  Indice : [[909 662 547 799 431  25]]
[5.55111512e-16 5.88137910e-01 6.26111225e-01 6.26262626e-01
 6.26469514e-01 6.26602733e-01]
6
Recommendation for movie_id
1       0.0
2       0.0
3       2.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      4.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      4.0
24      3.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655    0.0
1656    0.0
1658    0.0
1659    0.0
1660    0.0
1661    0.0
1662    0.0
1663    0.0
1664    0.0
1665    0.0
1668    0.0
1669    0.0
1670    0.0
1671    0.0
1672    0.0
1674    0.0
1677    0.0
1678    0.0
1680    0.0
1681    0.0
1682    0.0



1: 64, with distance of 0.5097108855920462:
2: 886, with distance of 0.5249869247045857:
3: 653, with distance of 0.5251268907544304:
4: 87, with distance of 0.5252129634951446:
5: 198, with distance of 0.5275302928289775:
[64, 886, 653, 87, 198]
Distance : [[4.44089210e-16 5.61026547e-01 5.85730676e-01 5.94933303e-01
  5.99461218e-01 6.02134154e-01]]  Indice : [[933  84 715 150 693   6]]
[4.44089210e-16 5.61026547e-01 5.85730676e-01 5.94933303e-01
 5.99461218e-01 6.02134154e-01]
6
Recommendation for movie_id
1       2.0
2       4.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      4.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
1644    0.0
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1651    0.0
1652    0.0
1653    0.0
1654    0.0
1655